In [7]:
using Random
using Plots
using ProgressMeter
using Colors

# f(x) = 5 - 24x + 17x^2 - (11/3)x^3 + (1/4)x^4
f(x) = (x-5)*(x-2)*(x+5)^2

function decimal_to_binary(x::Int, length::Int)
    binary_str = bitstring(x)[end-length+1:end]
    return binary_str
end

function binary_to_decimal(binary_str::String)
    return parse(Int, binary_str, base=2)
end

function calculate_fitness(population::Vector{String}, fitness_func)
    fitness_values = Float64[]
    decimal_values = Int[]
    for chrom in population
        x = binary_to_decimal(chrom)
        push!(decimal_values, x)
        push!(fitness_values, fitness_func(x))
    end
    return fitness_values, decimal_values
end

function select_parents(population::Vector{String}, fitness_values::Vector{Float64})
    inverted_fitness = maximum(fitness_values) .- fitness_values .+ 1e-10
    total_fitness = sum(inverted_fitness)
    probabilities = inverted_fitness ./ total_fitness
    
    parent1_idx = findfirst(cumsum(probabilities) .≥ rand())[1]
    parent2_idx = findfirst(cumsum(probabilities) .≥ rand())[1]
    
    return population[parent1_idx], population[parent2_idx]
end

function crossover(parent1::String, parent2::String, crossover_rate::Float64)
    if rand() > crossover_rate
        return parent1, parent2
    end
    
    crossover_point = rand(1:length(parent1)-1)
    child1 = parent1[1:crossover_point] * parent2[crossover_point+1:end]
    child2 = parent2[1:crossover_point] * parent1[crossover_point+1:end]
    
    return child1, child2
end

function mutate(child::String, mutation_rate::Float64)
    if rand() > mutation_rate
        return child
    end
    
    mutation_point = rand(1:length(child))
    mutated_child = collect(child)
    mutated_child[mutation_point] = mutated_child[mutation_point] == '0' ? '1' : '0'
    return join(mutated_child)
end

function plot_generation(x_values::Vector{Int}, fitness_values::Vector{Float64}, 
                        generation::Int, x_range::Tuple{Int,Int}, best_x::Int)
    x_plot = range(x_range[1], x_range[2], length=100)
    y_plot = f.(x_plot)
    
    true_min_x = argmin(y_plot)
    true_min_y = y_plot[true_min_x]
    
    p = plot(x_plot, y_plot, label="f(x)", linewidth=2, color=:blue, 
             title="Поколение $generation", xlabel="x", ylabel="f(x)")
    
    scatter!(x_values, fitness_values, color=:red, markersize=8, 
             label="Популяция", marker=:circle)
    
    best_y = f(best_x)
    scatter!([best_x], [best_y], color=:green, markersize=10, 
             label="Лучшая точка", marker=:star5)
    
    scatter!([x_plot[true_min_x]], [true_min_y], color=:purple, markersize=8, 
             label="Истинный минимум", marker=:diamond)
    
    return p
end

function genetic_algorithm_with_animation(fitness_func, population_size::Int, 
                                        chromosome_length::Int, generations::Int, 
                                        crossover_rate::Float64, mutation_rate::Float64,
                                        x_range::Tuple{Int,Int}, gif_filename::String)

    min_x, max_x = x_range
    initial_population = [decimal_to_binary(rand(min_x:max_x), chromosome_length) 
                          for _ in 1:population_size]
    
    current_population = copy(initial_population)
    animation = Animation()
    best_x = 0
    best_fitness = Inf
    
    for gen in 1:generations
        fitness_values, decimal_values = calculate_fitness(current_population, fitness_func)
        
        current_best_idx = argmin(fitness_values)
        if fitness_values[current_best_idx] < best_fitness
            best_x = decimal_values[current_best_idx]
            best_fitness = fitness_values[current_best_idx]
        end
        
        p = plot_generation(decimal_values, fitness_values, gen, x_range, best_x)
        frame(animation, p)
        
        println("\nПоколение $gen:")
        println("Хромосома (двоичная) | Десятичное значение | Приспособленность")
        println("---------------------|---------------------|------------------")
        for (chrom, x, fit) in zip(current_population, decimal_values, fitness_values)
            println("$chrom | $x | $fit")
        end
        
        new_population = String[]
        
        while length(new_population) < population_size
            parent1, parent2 = select_parents(current_population, fitness_values)
            child1, child2 = crossover(parent1, parent2, crossover_rate)
 
            child1 = mutate(child1, mutation_rate)
            child2 = mutate(child2, mutation_rate)
            
            push!(new_population, child1)
            if length(new_population) < population_size
                push!(new_population, child2)
            end
        end
        
        current_population = new_population
    end
    
    gif(animation, gif_filename, fps=1)
    
    final_fitness, final_decimal = calculate_fitness(current_population, fitness_func)
    best_idx = argmin(final_fitness)
    best_chrom = current_population[best_idx]
    best_x = final_decimal[best_idx]
    best_fitness = final_fitness[best_idx]
    
    println("\nРезультат:")
    println("Лучшая хромосома: $best_chrom")
    println("Десятичное значение: $best_x")
    println("Приспособленность: $best_fitness")
    
    return best_x
end

population_size = 8
chromosome_length = 6
generations = 30
crossover_rate = 0.8
mutation_rate = 0.1
x_range = (0, 10)
gif_filename = "genetic_algorithm_animation.gif"

best_solution = genetic_algorithm_with_animation(
    f, population_size, chromosome_length, generations, 
    crossover_rate, mutation_rate, x_range, gif_filename
)

println("\nАнимация сохранена в файл: $gif_filename")


Поколение 1:
Хромосома (двоичная) | Десятичное значение | Приспособленность
---------------------|---------------------|------------------
000100 | 4 | -162.0
000111 | 7 | 1440.0
000011 | 3 | -128.0
000111 | 7 | 1440.0
000111 | 7 | 1440.0
000101 | 5 | 0.0
001001 | 9 | 5488.0
001010 | 10 | 9000.0

Поколение 2:
Хромосома (двоичная) | Десятичное значение | Приспособленность
---------------------|---------------------|------------------
000001 | 1 | 144.0
000111 | 7 | 1440.0
000111 | 7 | 1440.0
000111 | 7 | 1440.0
000111 | 7 | 1440.0
000011 | 3 | -128.0
000111 | 7 | 1440.0
000101 | 5 | 0.0

Поколение 3:
Хромосома (двоичная) | Десятичное значение | Приспособленность
---------------------|---------------------|------------------
000011 | 3 | -128.0
000101 | 5 | 0.0
000011 | 3 | -128.0
000101 | 5 | 0.0
000011 | 3 | -128.0
000111 | 7 | 1440.0
000011 | 3 | -128.0
000011 | 3 | -128.0

Поколение 4:
Хромосома (двоичная) | Десятичное значение | Приспособленность
---------------------|-------------

┌ Info: Saved animation to c:\Users\Golum\Desktop\iu9-education\optimization-methods\let5\genetic_algorithm_animation.gif
└ @ Plots C:\Users\Golum\.julia\packages\Plots\kLeqV\src\animation.jl:156
